In [1]:
import numpy as np
import pandas as pd

In [2]:
def calc_epsilon_s(n):
  # Calculate epsilon_s in percentage
  # given the number of significant figures
  return 0.5 * 10**(2-n)

def error(new, old):
  # In percentage
  return abs((new-old)/new)*100

In [3]:
def func(new, old):
    return new

def relax(new, old, l):
    return new*l+(1-l)*old

#Gauss Seidel

In [4]:
def GaussSeidel(A, b, f, digit=4):
  '''
  GaussSeidel: Gauss Seidel method
    GaussSeidel(A,b, digit): Gauss Seidel without relaxation

  Input:
    A = coefficient matrix
    b = right hand side vector
    digit = significant figure accuracy

  Output:
    DataFrame of [Iteration, x1...xn , ea1...ean] per iteration
  '''

  # coefficient matrix dimention
  m,n = np.shape(A)
  if m != n:
    print('Matrix A must be square')
    return

  # es = stop criterion (default = epsilon_s of significant figure accuracy(digit))
  es = calc_epsilon_s(digit)
  result = {"Iteration":[]}

  # generate keys for dictionary(x1...xn, ea1...ean)
  x = np.zeros(n)
  x_key = []
  ea_key = []
  for i in range(n):
    x_key.append("x" + str(i+1))
    ea_key.append("ea_" + str(i+1))
    result[x_key[i]] = []
  for i in range(n):
    result[ea_key[i]] = []

  # solve each of the equations for its unknown on the diagonal
  C = np.copy(A) * 1.0
  d = np.copy(b) * 1.0
  for i in range(n):
    C[i,i] = 0
    d[i] = d[i]/A[i,i]
    C[i] = C[i]/A[i,i]


  # for checking all ea satisfy stopping condition
  tolerated = False
  i = 1

  # loop per iteration
  while(not tolerated):
    iter_result = [i]
    result["Iteration"].append(i)
    tolerated = True # all ea satisfy stopping condition until proven false
    for j in range(n): # loop for each x
      new_x = f(d[j] - (x * C[j]).sum(), x[j])
      err = error(new_x, x[j])
      x[j] = new_x
      result[ea_key[j]].append(err)
      result[x_key[j]].append(x[j])
      if(err > es):
        tolerated = False
    if(tolerated):
      break
    i = i + 1

  return pd.DataFrame(result)

In [9]:
# # Coeficient Matrix
# A = np.array(
#     [
#         [3,-0.1,-0.2],
#         [0.1,7,-0.3],
#         [0.3,-0.2,10]
#     ])
# # Right Hand Side Vector
# b = np.array([7.85,-19.3,71.4])
# GaussSeidel(A, b, func, 5).style.hide(axis="index")

# Diagonal dominance, diagonal absolute value for given row > sum of other coefficient in the same row
A = np.array(
    [
        [10, -2],
        [-3, 12],
    ]
)
b = np.array([8, 9])
f = lambda old, new: relax(old, new, 1.2)
GaussSeidel(A, b, f, 2).style.hide(axis="index")

Iteration,x1,x2,x3,ea_1,ea_2,ea_3
1,25.433333,3.710909,0.260482,100.000000,100.000000,100.000000
2,13.768307,5.826671,0.429563,84.723757,36.311674,39.361111
3,6.928995,7.000243,0.544076,98.705673,16.764741,21.047250
4,2.945361,7.618227,0.625847,135.251168,8.111905,13.065630
5,0.650836,7.909489,0.687813,352.550195,3.682441,9.009151
6,-0.645342,8.009495,0.737648,200.851334,1.248586,6.755948
7,-1.352178,7.998397,0.779909,52.273865,0.138748,5.418634
8,-1.711821,7.923598,0.817304,21.009378,0.944003,4.575462
9,-1.867615,7.813106,0.851450,8.341871,1.414189,4.010387
10,-1.904394,7.683446,0.883316,1.931274,1.687516,3.607474


#Jacobi

In [6]:
def Jacobi(A, b, f, digit=4):
  '''
  Jacobi: Jacobi method
    Jacobi(A,b, digit): Jacobi without relaxation

  Input:
    A = coefficient matrix
    b = right hand side vector
    digit = significant figure accuracy

  Output:
    DataFrame of [Iteration, x1...xn , ea1...ean] per iteration
  '''

  # coefficient matrix dimention
  m,n = np.shape(A)
  if m != n:
    print('Matrix A must be square')
    return

  # es = stop criterion (default = epsilon_s of significant figure accuracy(digit))
  es = calc_epsilon_s(digit)
  result = {"Iteration":[]}

  # generate keys for dictionary(x1...xn, ea1...ean)
  x = np.zeros(n)
  x_key = []
  ea_key = []
  for i in range(n):
    x_key.append("x" + str(i+1))
    ea_key.append("ea_" + str(i+1))
    result[x_key[i]] = []
  for i in range(n):
    result[ea_key[i]] = []

  # solve each of the equations for its unknown on the diagonal
  C = np.copy(A) * 1.0
  d = np.copy(b) * 1.0
  for i in range(n):
    C[i,i] = 0
    d[i] = d[i]/A[i,i]
    C[i] = C[i]/A[i,i]

  # for checking all ea satisfy stopping condition
  tolerated = False
  i = 1
  # temporary place for newly calculated x
  new_x = np.zeros(n)

  # loop per iteration
  while(not tolerated):
    iter_result = [i]
    result["Iteration"].append(i)
    tolerated = True # all ea satisfy stopping condition until proven false
    for j in range(n): # calculate all new x first before changing the previous x
      new_x[j] = f(d[j] - (x * C[j]).sum(), x[j])
    for j in range(n):
      err = error(new_x[j], x[j])
      x[j] = new_x[j]
      result[ea_key[j]].append(err)
      result[x_key[j]].append(x[j])
      if(err > es):
        tolerated = False
    if(tolerated):
      break
    i = i + 1
  return pd.DataFrame(result)

In [7]:
# Coeficient Matrix
A = np.array(
    [
        [3,-0.1,-0.2],
        [0.1,7,-0.3],
        [0.3,-0.2,10]
    ])
# Right Hand Side Vector
b = np.array([7.85,-19.3,71.4])
Jacobi(A, b, func, 5).style.hide(axis="index")

Iteration,x1,x2,x3,ea_1,ea_2,ea_3
1,2.616667,-2.757143,7.140000,100.000000,100.000000,100.000000
2,3.000762,-2.488524,7.006357,12.799924,10.794313,1.907451
3,3.000806,-2.499738,7.000207,0.001481,0.448632,0.087861
4,3.000022,-2.500003,6.999981,0.026128,0.010569,0.003223
5,2.999999,-2.500001,6.999999,0.000795,0.000061,0.000260
6,3.000000,-2.500000,7.000000,0.000042,0.000045,0.000011
